Weenies and Buns is a food processing plant which manufactures hot dogs and hot dog buns. They grind their own flour for the hot dog buns at a maximum rate of $200$ pounds per week. Each hot dog bun requires $0.1$ pound of flour. 

They currently have a contract with Pigland, Inc., which specifies that a delivery of $800$ pounds of pork product is delivered every Monday. Each hot dog requires $1/4$ pound of pork product. 

All the other ingredients in the hot dogs and hot dog buns are in plentiful supply. 

Finally, the labor force at Weenies and Buns consists of $5$ employees working full time (40 hours per week each). Each hot dog requires $3$ minutes of labor, and each hot dog bun requires $2$ minutes of labor. 

Each hot dog yields a profit of $\$0.80$, and each bun yields a profit of $\$0.30$. Weenies and Buns would like to know how many hot dogs and how many hot dog buns they should produce each week so as to achieve the highest possible profit.

In [1]:
from pyomo.environ import *

In [2]:
model = AbstractModel()

## Sets

In [3]:
model.products = Set()

## Parameters

In [4]:
model.products_profit = Param(model.products)
model.raw_material_available = Param(model.products)
model.raw_material_used_per_product = Param(model.products)
model.work_hours_per_unit = Param(model.products)
model.number_of_employees = Param()
model.work_hours_per_employee = Param()


## Variables

In [5]:
model.quotas = Var(model.products, within=NonNegativeReals)

## Objective

In [6]:
def total_profit(model):
    return summation(model.products_profit, model.quotas)


model.final_profit = Objective(rule=total_profit, sense=maximize)

## Constraints


In [7]:
def raw_material(model, product):
    return model.raw_material_used_per_product[product] * model.quotas[product] <= model.raw_material_available[product]
    
model.raw_material_constraint = Constraint(model.products, rule=raw_material)

In [8]:
def max_work_hours(model):
    return sum(model.work_hours_per_unit[product] * model.quotas[product] for product in model.products) <= model.number_of_employees * model.work_hours_per_employee


model.max_work_hours_constraint = Constraint(rule=max_work_hours)

## Instantiate the Model

In [9]:
data = { None : { 'products' : { None : ('Hot dog', 'Buns') }, 
                  'products_profit' : { 'Hot dog' : .80, 'Buns' : .30 },
                  'raw_material_available' : { 'Hot dog' : 800, 'Buns' : 200 },
                  'raw_material_used_per_product' : { 'Hot dog' : .25, 'Buns' : .10 },
                  'work_hours_per_unit' : { 'Hot dog' : 3/float(60), 'Buns' : 2/float(60) },
                  'number_of_employees' : {None : 5},
                  'work_hours_per_employee' : {None : 40}
                }
        }

In [10]:
instance = model.create_instance(data)
instance.pprint()

1 Set Declarations
    products : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Buns', 'Hot dog']

6 Param Declarations
    number_of_employees : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     5
    products_profit : Size=2, Index=products, Domain=Any, Default=None, Mutable=False
        Key     : Value
           Buns :   0.3
        Hot dog :   0.8
    raw_material_available : Size=2, Index=products, Domain=Any, Default=None, Mutable=False
        Key     : Value
           Buns :   200
        Hot dog :   800
    raw_material_used_per_product : Size=2, Index=products, Domain=Any, Default=None, Mutable=False
        Key     : Value
           Buns :   0.1
        Hot dog :  0.25
    work_hours_per_employee : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    40
    work_hours_per_unit : Size=2, Index=products, Domain=Any, Default=None, Mutable=False
       

## Solve the problem

In [11]:
opt = SolverFactory('glpk')

In [12]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 2920.0
  Upper bound: 2920.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011947154998779297
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: feasible
  Message: None
  Objective:
    final_profit:
      Value: 2920
  Variable:
    quotas[Buns]:
      Value: 1200
    quotas[Hot dog]:
      Value: 3200
  Constraint: No values

